# Simple state space model for voting intention (using pymc3)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-setup" data-toc-modified-id="Python-setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python setup</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#A-Bayesian-poll-aggregation-model-(pooling-polls)" data-toc-modified-id="A-Bayesian-poll-aggregation-model-(pooling-polls)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>A Bayesian poll aggregation model (pooling polls)</a></span><ul class="toc-item"><li><span><a href="#Supporting-functions" data-toc-modified-id="Supporting-functions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Supporting functions</a></span></li><li><span><a href="#Analysis-frame" data-toc-modified-id="Analysis-frame-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Analysis frame</a></span></li><li><span><a href="#And-do-it-..." data-toc-modified-id="And-do-it-...-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>And do it ...</a></span></li></ul></li><li><span><a href="#Exploratory" data-toc-modified-id="Exploratory-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exploratory</a></span></li><li><span><a href="#Other" data-toc-modified-id="Other-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Other</a></span></li></ul></div>

## Python setup

In [1]:
# data science imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# pymc probabilistic programming
import arviz as az
import pymc as pm

# system imports
from pathlib import Path
from io import StringIO

# local imports
import common

In [2]:
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue Apr 19 2022 13:19:16

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.2.0

pandas    : 1.4.2
matplotlib: 3.5.1
arviz     : 0.12.0
numpy     : 1.22.1
pymc      : 4.0.0b5

Watermark: 2.3.0



In [3]:
# pymc3 set-up
az.style.use("arviz-darkgrid")

In [4]:
# plotting stuff
plt.style.use('./markgraph.mplstyle')

In [5]:
# path to the model data 
DATA_DIR = '../data/'
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

In [6]:
CHART_DIR = '../charts/'
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)

## Load data

In [7]:
# Analysis based on this data ...
data_file = f'{DATA_DIR}primary_vote.csv' # f'{DATA_DIR}two_party_preferred.csv'

# Minimum polls requirement for a pollster
MIMIMUM_POLLS_FROM_POLLSTER = 2
# The one ANU poll since 2019 look like a rogue outlier, so let's exclude it

# Plot left-side footnotes
LFOOTER = (
    'Polling data from Wikipedia; '
    f'Pollsters ignored with < {MIMIMUM_POLLS_FROM_POLLSTER} polls'
)

In [8]:
# Check the data file has been updated today
common.check_file_current(data_file, 
    'Make sure you run the data capture notebook first')

In [9]:
# read in 2pp data; remind me, what are the column names ...
tpp = pd.read_csv(data_file, parse_dates=['Mean Date'], index_col=0)
tpp['Mean Date'] = pd.to_datetime(tpp['Mean Date'])
tpp = tpp.sort_values('Mean Date')
tpp.columns

Index(['Primary vote L/NP', 'Primary vote ALP', 'Primary vote GRN',
       'Primary vote ONP', 'Primary vote UAP', 'Primary vote OTH', 'Date',
       'Mean Date', 'Brand', 'Interview mode', 'Sample size',
       'Calculated 2pp L/NP', 'Calculated 2pp ALP'],
      dtype='object')

In [10]:
# Combine the other primary vote in the main dataframe
OTHER = 'Other Combined Primary'
tpp[OTHER] = tpp[['Primary vote ONP', 'Primary vote UAP', 'Primary vote OTH']].fillna(0).sum(axis=1)

In [11]:
# Number of polls
len(tpp)

140

In [12]:
# Number of polls from each pollster
brand_counts = tpp['Brand'].value_counts()
brand_counts

Essential            48
Newspoll-YouGov      47
Roy Morgan           31
Resolve Strategic    12
ANU                   1
Ipsos                 1
Name: Brand, dtype: int64

In [13]:
# Remove pollsters with minimal polling history
keep = brand_counts[brand_counts >= MIMIMUM_POLLS_FROM_POLLSTER].index
tpp = tpp[tpp['Brand'].isin(keep)]
tpp['Brand'].value_counts()

Essential            48
Newspoll-YouGov      47
Roy Morgan           31
Resolve Strategic    12
Name: Brand, dtype: int64

In [14]:
len(tpp)

138

## A Bayesian poll aggregation model (pooling polls)

### Supporting functions

In [15]:
def prepare_data_for_analysis(df, column):
    
    # make sure data is in date order
    assert df['Mean Date'].is_monotonic_increasing, 'Data must be in ascending date order'

    # get our zero centered observations
    y = df[column]
    centre_offset = -y.mean()
    zero_centered_y = y + centre_offset
    n_polls = len(zero_centered_y),
    
    # get our day-date mapping
    day_zero = df['Mean Date'].min()
    n_days = int((df['Mean Date'].max() - day_zero) / pd.Timedelta(days=1)) + 1
    poll_day = ((df['Mean Date'] - day_zero) / pd.Timedelta(days=1)).astype(int)
    
    # get our poll-branding information
    poll_brand = df.Brand.astype('category').cat.codes
    n_brands = len(poll_brand.unique())
    brand_map = {code: brand for brand, code in zip(df.Brand, poll_brand)}

    # measurement error 
    assumed_sample_size = 1000
    measurement_error_sd = np.sqrt((50 * 50) / assumed_sample_size)

    return (zero_centered_y, centre_offset, n_polls, n_days, day_zero, 
            poll_day, poll_brand, brand_map, n_brands, measurement_error_sd)

In [16]:
def define_model(n_brands, n_days, poll_brand, 
                 zero_centered_y, 
                 measurement_error_sd):
    """PyMC3 model for pooling/aggregating voter opinion polls"""
    
    model = pm.Model()
    with  model:
        # priors 
        unanchored_house_bias = pm.Cauchy("unanchored_house_bias", 
                                          alpha=0, beta=10, shape=n_brands)
        zero_sum_house_bias = pm.Deterministic('zero_sum_house_bias', 
            var=(unanchored_house_bias - unanchored_house_bias.mean()))
        
        # temporal model
        EARLY_DATA_ITEMS = 5 
        educated_guess = zero_centered_y[:min(EARLY_DATA_ITEMS, len(zero_centered_y))].mean()
        SPAN = 2.0
        starting_distro = pm.Normal('starting_distro', mu=educated_guess, sigma=SPAN) 
        DRIFT = 0.0
        innovation = 0.15 # from experience ... day-to-day change distribution sigma
        grw = pm.GaussianRandomWalk('grw', mu=DRIFT, sigma=innovation, 
                                    init=starting_distro, steps=n_days) ### FAILS HERE
    
        # the observational model
        observed = pm.Normal("observed", 
                             mu=grw[poll_day] 
                                + zero_sum_house_bias[poll_brand.to_list()],
                             sigma=measurement_error_sd, observed=zero_centered_y)
    return model

In [17]:
def draw_samples(model):
    with model:
        trace = pm.sample(1000, tune=1000, progressbar=True, 
                          return_inferencedata=True)
        summary = az.summary(trace) # used below
        az.plot_trace(trace)      
        
    return trace, summary

In [18]:
def get_var_as_frame(inferencedata, variable_name):
    """Return a dataframe - column(s) are variables - rows are draws"""
    return pd.DataFrame(
        inferencedata
        .posterior
        .stack(draws=("chain", "draw"))[variable_name]
        .values
        .T
    )


def quants_and_mean(frame, quants):
    """Return a DataFrame - columns are quants and mean - rows are variable"""
    results = frame.quantile(quants).T
    results['mean'] = frame.mean()
    return results


def get_quant_iterator(quants):
    """get iterator for paired quantiles, with matching labels"""
    length = len(quants)
    assert length > 0, 'Quantile list should not be empty'
    assert (length % 2 == 0), 'Quantile list must have an even number of members'
    assert pd.Series(quants).is_monotonic_increasing, 'Qantiles must be ascending ordered'
    middle = int(length / 2)
    start = quants[:middle]
    stop = quants[-1:(middle-1):-1]
    label = (
        ((pd.Series(stop) - pd.Series(start)) * 100)
        .round(0)
        .astype(int)
        .astype(str)
        + '% HDI'
    ).to_list()
    return zip(start, stop, label)

In [19]:
QUANTS = (0.025, 0.1, 0.25, 0.75, 0.9, 0.975)
def plot_aggregation(trace, df, column, day_zero, n_days, centre_offset, 
                     point_color, line_color, title):
    
    # get the data
    grw = get_var_as_frame(trace, 'grw') - centre_offset
    grw.columns = pd.date_range(start=day_zero, periods=n_days)
    grw_summary = quants_and_mean(grw, QUANTS)
    
    # plot
    fig, axes = common.initiate_plot()
    axes.plot(grw_summary.index, grw_summary['mean'], 
            lw=2.0, c=line_color, 
            label='Mean Estimated Voting Intention')
    alpha = 0.1
    for start, stop, label in get_quant_iterator(QUANTS):
        axes.fill_between(x=grw_summary.index, y1=grw_summary[start], 
                        y2=grw_summary[stop], color=line_color, 
                        alpha=alpha, label=label)
        alpha += 0.1
    common.annotate_endpoint(axes, grw_summary['mean'])

    common.add_data_points_by_pollster(ax=axes, df=df, column=column,
                                      p_color=point_color, )
    common.add_h_refence(axes, 50)
    axes.yaxis.set_major_formatter(mtick.PercentFormatter())
    
    lfooter = LFOOTER
    if '2pp' in title:
        lfooter += '; 2pp calculated from primary votes and 2019 preference flows'
    axes.legend(loc='best', ncol=2)
    full_title = f'Gaussian Random Walk - {title} Voting Intention'
    common.plot_finalise(axes,
                         title=full_title,
                         lfooter=lfooter)

In [20]:
def plot_house_effects(trace, brand_mapping, title, point_color, line_color):
    
    # get the relevant data
    zshb = get_var_as_frame(trace, 'zero_sum_house_bias')
    print(zshb.columns)
    zshb.columns =  zshb.columns.map(brand_mapping)
    zshb_summary = quants_and_mean(zshb, QUANTS)

    # and plot it
    fig, ax = common.initiate_plot()
    alpha = 0.1
    for start, stop, label in get_quant_iterator(QUANTS):
        ax.barh(zshb_summary.index, 
                width=zshb_summary[stop]-zshb_summary[start], 
                left=zshb_summary[start], 
                color=line_color, alpha=alpha, label=label)
        alpha += 0.1
    ax.scatter(zshb_summary['mean'], zshb_summary.index, 
               s=50, marker='o', color=point_color, label='Mean estimate')
    xlabel = f'Relative percentage point bias to {title.split()[0]}'
    full_title = f'Zero Sum Pollster Effects - {title}'
    ax.legend(loc='best')
    common.plot_finalise(ax,
                         title=full_title,
                         xlabel=xlabel,
                         lfooter=LFOOTER)

### Analysis frame

In [21]:
analytic_frame = f"""
column, title, point_color, line_color
Calculated 2pp L/NP, Coalition 2pp,     {common.P_COLOR_COALITION}, {common.COLOR_COALITION}
Primary vote L/NP,   Coalition Primary, {common.P_COLOR_COALITION}, {common.COLOR_COALITION}
Primary vote ALP,    Labor Primary,     {common.P_COLOR_LABOR},     {common.COLOR_LABOR}
Primary vote GRN,    Greens Primary,    {common.P_COLOR_GREEN},     {common.COLOR_GREEN}
{OTHER},             Others Primary,    {common.P_COLOR_OTHER},     {common.COLOR_OTHER}

"""

analytic_frame = pd.read_csv(StringIO(analytic_frame), skipinitialspace=True)
analytic_frame

,column,title,point_color,line_color
0,Calculated 2pp L/NP,Coalition 2pp,#0000dd,darkblue
1,Primary vote L/NP,Coalition Primary,#0000dd,darkblue
2,Primary vote ALP,Labor Primary,#dd0000,darkred
3,Primary vote GRN,Greens Primary,green,darkgreen
4,Other Combined Primary,Others Primary,orange,darkorange


### And do it ...

In [22]:
for index, series in analytic_frame.iterrows():
    print(series['title'])
    
    (zero_centered_y, centre_offset, n_polls, n_days, day_zero, poll_day, 
        poll_brand, brand_map, n_brands, measurement_error_sd) = (
        prepare_data_for_analysis(tpp, series['column']))
    
    model = define_model(n_brands, n_days, poll_brand, zero_centered_y,
                         measurement_error_sd)
        
    trace, summary = draw_samples(model)
    
    plot_aggregation(trace, tpp, series['column'], day_zero, n_days, 
                     centre_offset, series['point_color'], 
                     series['line_color'], series['title'])
    
    plot_house_effects(trace, brand_map, series['title'], 
                       series['point_color'], series['line_color'])

Coalition 2pp


TypeError: dist() missing 1 required positional argument: 'dist_params'

In [ ]:
# I really like this feature ...
pm.model_to_graphviz(model)

## Exploratory

## Other

In [ ]:
common.print_warnings()